# Record city cameras testing

#### Import modules

In [3]:
from time import time
from IPython.display import clear_output as co

#### Change directory to cams-rio api project

In [4]:
cd ../../Apps/APIs/cams-rio

C:\Users\luisr\Desktop\Repositories\Apps\APIs\cams-rio


#### Import recorder class

In [ ]:
from rec import Recorder

#### Recorder class instance

In [ ]:
rec = Recorder(retries=10, skip_method=None, skip_max=15)

---
#### Loop until first working camera

In [11]:
res = {'message': 'FAILED'}
code = 0
while res['message'] == 'FAILED':
    res = rec.record(code=code, folder='tests/', n_frames=15, saveas='video-from-list')
    code += 1
    print(res)

---
## Record single camera

Camera with "overread" error: 538

#### Record using python recorder module

In [13]:
code = 2

res = rec.record(code=code, folder='tests', n_frames=15, saveas='video')

UPLOAD SUCCESS. CODE 2, FRAMES 15, STARTED AT 2023-04-07 09-42-15, FINISHED AT 2023-04-07 09-42-20


#### Record using api http request

In [13]:
code = 2

res = rec.record(code=code, folder='tests', n_frames=15, saveas='video')

UPLOAD SUCCESS. CODE 2, FRAMES 15, STARTED AT 2023-04-07 09-42-15, FINISHED AT 2023-04-07 09-42-20


#### Compare performance for different recording methods

In [16]:
def exec_secs(func, n=100, decimals=4):
    s = time()
    for i in range(n): func()
    return round((time() - s) / n, decimals)

code = 2

def func1():
    rec.record(code=code, folder='tests', n_frames=6, saveas='video')
def func2():
    rec.record(code=code, folder='tests', n_frames=6, saveas='video-from-list')
    

t1 = exec_secs(func1, n=10)
t2 = exec_secs(func2, n=10)

co(True); print(f'\n* T1: {t1} · T2: {t2}')

---
## Record Cameras Around Collection of Points (using http)

#### Example http request parameters

In [ ]:
import json, requests

data_river = {
  'n': 1,
  'radius': 50.0,
  'n_frames': 45,
  'saveas': 'video',
  'workers': None,
  'urls': [{
    'url': 'cameras',
#     'timestamp': 'timstamp',
#     'max_age': '1H', # should be present in url object or in query object only
    'query': [{
        'query': {'Codigo': [3318, 3323, 1487]},
        'prefix': 'rivers',
#         'max_age': '1H',
    },]
}]
}

data_river_list = {**data_river, 'saveas': 'video-from-list'}

data_rain = {
  'n': 3,
 'radius': 250.0,
 'n_frames': 45,
 'saveas': 'video',
 'workers': None,
 'urls': [{'url': 'mongo/Cameras AlertaRio',
   'query': [
    {'query': {'Codigo': [2]}, 'prefix': 'rivers'},
]}]}

#### Convert parameters to json format (optional)

In [58]:
print(json.dumps(data_river))

{"n": 1, "radius": 50.0, "n_frames": 45, "saveas": "video", "workers": null, "urls": [{"url": "cameras", "query": [{"query": {"Codigo": [3318, 3323, 1487]}, "prefix": "rivers"}]}]}


#### Record points using HTTP

In [53]:
res = requests.post('http://127.0.0.1:5001/record/points', json=data_river_list)
# res = requests.post('https://cams.octacity.dev/record/points', json=data_river)

res.reason, res.text

---
## Record many cameras

#### Record many parameters

In [78]:
data_river = {
    'workers': None,
    'params': [{
        'code': 3318,
        'folder': 'rivers/manual',
        'n_frames': 45,
        'saveas': 'video',
    }, {
        'code': 3323,
        'folder': 'rivers/manual',
        'n_frames': 45,
        'saveas': 'video',
    },{
        'code': 1487,
        'folder': 'rivers/manual',
        'n_frames': 45,
        'saveas': 'video',
    }]
}

#### http request

In [78]:
res = requests.post('http://127.0.0.1:5001/record/many', json=data_many)

res.reason, res.text

---
## Record Cameras by Trigger

#### Trigger parameters

In [10]:
data_trigger_rain = {
    'url': 'mongo/Cameras AlertaRio',
    'triggers': [
        'acumulado_chuva_5_min_1',
        'acumulado_chuva_5_min_2',
        'acumulado_chuva_5_min_3'
    ],
    'logic': 'all',
    'merge': False,
    'normal': False,
    'folder': 'rain',
    'n_frames': 45,
    'saveas': 'video',
    'workers': 350
}

data_trigger_normal = {
    'url': 'mongo/Cameras AlertaRio',
    'triggers': [
        'acumulado_chuva_15_min_1',
        'acumulado_chuva_15_min_2',
        'acumulado_chuva_15_min_3'
    ],
    'logic': 'any',
    'merge': False,
    'normal': True,
    'folder': 'normal',
    'n_frames': 45,
    'saveas': 'video',
    'workers': 350
}

data_trigger_polygons = {
    'url': 'mongo/Polygons AlertaRio',
    'triggers': [
        'acumulado_chuva_15_min_1',
        'acumulado_chuva_15_min_2',
        'acumulado_chuva_15_min_3'
    ],
    'logic': 'any',
    'merge': True,
    'normal': True,
    'folder': 'polygons/normal',
    'n_frames': 45,
    'saveas': 'video',
    'workers': None
}

#### Count number of cameras triggered

In [11]:
from modules.trigger import Trigger

# data = data_trigger_polygons
data = data_trigger_rain

cams = Trigger(data['url'], data['triggers'], data['logic'], data['merge'])

print(f'\n* IN-ALERT: {len(cams.in_alert)} · NORMAL: {len(cams.normal)}')


* IN-ALERT: 356 · NORMAL: 2480


#### Record cameras by trigger

In [ ]:
res = requests.post('http://127.0.0.1:5001/record/trigger', json=data_trigger_polygons)
# res = requests.post('https://cams.octacity.dev/record/trigger', json=data_trigger_cams)
# res = requests.post('https://cams.octacity.dev/record/trigger', json=data_trigger_polygons)

res.reason, res.text

#### Compare HTTP performance

In [32]:
def func1():
    res = requests.post('http://127.0.0.1:5001/record/points', json=data)
def func2():
    res = requests.post('http://127.0.0.1:5001/record/points', json=data_list)
    

t1 = exec_secs(func1, n=10)
t2 = exec_secs(func2, n=10)

co(True); print(f'\n* T1: {t1} · T2: {t2}')


* T1: 26.5 · T2: 0.0
